In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import testgen
import importlib
import numpy as np
import os
import gdal
importlib.reload(testgen)

def save_as_gdal_dataset(fname,fname_orig,fname_pair,arr,pg,band_types):
    
    x_pixels = arr.shape[1]
    y_pixels = arr.shape[0]
    n_bands = arr.shape[2]
    driver = gdal.GetDriverByName('GTiff')
    ds = driver.Create(fname,x_pixels,y_pixels,n_bands,int(band_types[0]) )
    ds.SetGeoTransform(pg.makeGeoTransform())
    ds.SetProjection(pg.projection)
    ds.SetMetadata({"Original file name":fname_orig,"pair file name":fname_pair})
    for i in range(n_bands):
        band_i = i+1
        ds.GetRasterBand(band_i).WriteArray(arr[:,:,i])

    ds.FlushCache()
    del ds

In [6]:
samplegen = testgen.TestSampleGen_from_csv('test_files.csv')

im1_prev = None
im2_prev = None
im1 = None
im2 = None

gen = samplegen.most_alike()

sample_count = 0
for (im1,im2,pg,bt1,bt2),f1,f2 in gen:
    
#     print(os.path.split(f1)[1] )
#     imshow(im1[:,:,0],cmap='magma');
#     show()
#     print(os.path.split(f2)[1] )
#     imshow(im2[:,:,0],cmap='magma');
#     show()

    new_pair = True
    if (im1_prev is not None) and (im2_prev is not None):
        prev_stats = np.array([im1_prev.std()+im2_prev.std(),im1_prev.mean()+im2_prev.mean()])
        cur_stats = np.array([im1.std()+im2.std(),im1.mean()+im2.mean()])

        diff = np.abs(prev_stats-cur_stats)
        if diff[0] < 800 or diff[1] < 800:
            new_pair = False

    if new_pair:
        sample1_name = 'Samples/sample_{}_1.tiff'.format(sample_count)
        sample2_name = 'Samples/sample_{}_2.tiff'.format(sample_count)
        sample_count += 1
        
        save_as_gdal_dataset(sample1_name,f1,sample2_name,im1,pg,bt1)
        save_as_gdal_dataset(sample2_name,f2,sample1_name,im2,pg,bt2)
        print('Saving files: {} {}'.format(sample1_name,sample2_name))

    im1_prev = im1
    im2_prev = im2

Saving files: Samples/sample_0_1.tiff Samples/sample_0_2.tiff
Saving files: Samples/sample_1_1.tiff Samples/sample_1_2.tiff
Saving files: Samples/sample_2_1.tiff Samples/sample_2_2.tiff
Saving files: Samples/sample_3_1.tiff Samples/sample_3_2.tiff
Saving files: Samples/sample_4_1.tiff Samples/sample_4_2.tiff
Saving files: Samples/sample_5_1.tiff Samples/sample_5_2.tiff
Saving files: Samples/sample_6_1.tiff Samples/sample_6_2.tiff
Saving files: Samples/sample_7_1.tiff Samples/sample_7_2.tiff
Saving files: Samples/sample_8_1.tiff Samples/sample_8_2.tiff
Saving files: Samples/sample_9_1.tiff Samples/sample_9_2.tiff
Saving files: Samples/sample_10_1.tiff Samples/sample_10_2.tiff
Saving files: Samples/sample_11_1.tiff Samples/sample_11_2.tiff
Saving files: Samples/sample_12_1.tiff Samples/sample_12_2.tiff
Saving files: Samples/sample_13_1.tiff Samples/sample_13_2.tiff
Saving files: Samples/sample_14_1.tiff Samples/sample_14_2.tiff
Saving files: Samples/sample_15_1.tiff Samples/sample_15_2.t

KeyboardInterrupt: 

In [4]:
# write metadata file about test samples
import os
import gdal
import re
import csv

sample_directory = "/home/artef/Pictures/LS08Samples/"
# find maximum number of samples by parsing file names. Files are sample_SAMPLEID_SAMPLEPART.tiff
num_samples = max( [int(re.findall('_(\d+)_(\d+)',f)[0][0]) for f in os.listdir(sample_directory)] )+1
sample_files = [[None,None] for i in range(num_samples)]

for d in os.listdir(sample_directory):
    fname = os.path.join(sample_directory,d)
    sample_id, sample_part = re.findall('_(\d+)_(\d+)',d)[0]
    sample_files[int(sample_id)][int(sample_part)-1] = fname


with open('test_files_aligned.csv','w') as f:
    writer = csv.writer(f)
    # write header
    writer.writerow(["File1", "File2"])
    writer.writerows(sample_files)

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
np